In [47]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
from pandas.io.json import json_normalize

In [65]:
#Scrape clean data with Beautiful Soup and request for the url

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

page = requests.get(url).text

cdf = BeautifulSoup(page, 'lxml')

In [66]:
#retrieve only the postal codes table from the wikipedia page
datatable = cdf.find('table')
datatable;

In [67]:
#create a new pandas dataframe with the data from the wikipedia table 

headers = ['Postal Code', 'Borough', 'Neighborhood']
new_cdf = pd.DataFrame(columns=headers)
new_cdf;

In [68]:
#convert the table from html code to a pandas dataframe

for tr_cell in datatable.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        new_cdf.loc[len(new_cdf)] = row_data

In [69]:
new_cdf.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [70]:
#get rid of all the not assigned boroughs from our table 

new_cdf= new_cdf[new_cdf['Borough']!='Not assigned']

new_cdf.head(8)

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills


In [71]:
#check if any neighborhoods are not assigned inside an assigned borough
display = new_cdf.loc[new_cdf['Neighborhood']=='Not assigned']
display.head();

In [72]:
#appropriate code if a neighborhood was unassigned inside of a borough
#this line of code would make the not assigned neighborhood take on the name of the borough

new_cdf['Neighborhood'].replace('Not assigned',new_cdf['Borough'],inplace=True)
new_cdf.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [73]:
new_cdf.rename(columns={'Postal Code':'PostalCode'},inplace=True)
new_cdf.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [74]:
new_cdf.shape

(103, 3)

In [75]:
#loaded the csv file containing longitude and latitude 

cdf_cordinates = pd.read_csv('http://cocl.us/Geospatial_data')
cdf_cordinates.head(10)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [76]:
#used the join statement to add the latitude and longitude columns to the dataframe created in part 1

new_cdf = new_cdf.join(cdf_cordinates.set_index('Postal Code'), on='PostalCode')
new_cdf.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.753259,-79.329656
3,M4A,North York,Victoria Village,43.725882,-79.315572
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
